In [1]:
import pandas as pd
import json

### Read PUG_Prizes.xlsx sheet

In [2]:
# # Read PuG sheet and print information
# pug_df = pd.read_excel("./excel/PUG_Prizes.xlsx").astype(str)
# pug_df.info()
# pug_df["Type"].value_counts()

### Read PUG_2024_Programm_Talks.xlsx sheet

In [3]:
# Read program sheet and print information
program_df = pd.read_excel("./excel/PUG_2024_Programm_Talks_1504_MD_1804.xlsx").astype(str)
program_df.info()
program_df["Tag"].value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tag           182 non-null    object
 1   Session       182 non-null    object
 2   SessionStart  182 non-null    object
 3   SessionEnd    182 non-null    object
 4   TalkStart     182 non-null    object
 5   TalkEnd       182 non-null    object
 6   Talk          182 non-null    object
 7   Speaker       182 non-null    object
 8   Chair         182 non-null    object
 9   Inhalt        182 non-null    object
 10  Uni           182 non-null    object
 11  Abstract      182 non-null    object
dtypes: object(12)
memory usage: 17.2+ KB


Tag
Freitag       78
Donnerstag    76
Samstag       25
Mittwoch       3
Name: count, dtype: int64

### Initiate dict for PuG 2024 program

In [4]:
pug_days_list = program_df["Tag"].unique()
pug24_dict = dict.fromkeys(pug_days_list)
for key in pug24_dict.keys():
    s_dict = {"symposia": {}, "poster": {}, "keynote": {}}
    pug24_dict.update({key: s_dict})

### Put Keynotes into program dict

In [5]:
keynote = program_df.loc[program_df["Session"].str.startswith("Keyn")]

for _, k_row in keynote.iterrows():
    pug24_dict[k_row["Tag"]]["keynote"] = k_row.to_dict()

### Put Posters into program dict

In [6]:
poster = program_df.loc[program_df["Session"].str.startswith("Post")]

for _, p_row in poster.iterrows():
    pug24_dict[p_row["Tag"]]["poster"] = p_row.to_dict()	

### Put Symposia into program dict

In [7]:
# Read Symposia from Excel sheet and reset index
symposia = program_df.loc[program_df["Session"].str.startswith("Symp")]
symposia.reset_index(drop=True, inplace=True)
symposia_count = 1

# Iterate over symposia
for s_index, s_row in symposia.iterrows():
    s_group_index = "session" + s_row["Session"][-3:-2]
    s_id = "session" + s_row["Session"][-3:]
    s_talk = s_row["Talk"]
    s_title_index = "S" + (str(symposia_count) if symposia_count > 9 else "0" + str(symposia_count))
    s_title = s_title_index + " - " + s_talk
    s_date = s_row["Tag"] + " " + s_row["SessionStart"][:-3] + " - " + s_row["SessionEnd"][:-3]
    s_date_place = s_date + " | Ort TODO"
    s_chairs = {"author": s_row["Chair"], "uni": s_row["Uni"]}
    s_mantel = s_row["Inhalt"].replace("&", r"\&").replace("ε", r"$\epsilon$")
    s_mantel_brake = s_mantel.find("\n\n")
    if s_mantel_brake != -1:
        s_mantel = s_mantel[s_mantel_brake+2:]
    else:
        print("No brake found in mantel of symposium", s_title)

    # Add new symposia session if not present
    if s_group_index not in pug24_dict[s_row["Tag"]]["symposia"].keys():
        pug24_dict[s_row["Tag"]]["symposia"][s_group_index] = {}
    if s_id not in pug24_dict[s_row["Tag"]]["symposia"][s_group_index].keys():
        pug24_dict[s_row["Tag"]]["symposia"][s_group_index][s_id] = {}

    # Add Parameters to symposia session, if not present
    if "Index" not in pug24_dict[s_row["Tag"]]["symposia"][s_group_index][s_id]:
        symposia_count += 1
        pug24_dict[s_row["Tag"]]["symposia"][s_group_index][s_id]["Index"] = s_title_index
        pug24_dict[s_row["Tag"]]["symposia"][s_group_index][s_id]["Talk"] = s_talk
        pug24_dict[s_row["Tag"]]["symposia"][s_group_index][s_id]["Title"] = s_title
        pug24_dict[s_row["Tag"]]["symposia"][s_group_index][s_id]["DatePlace"] = s_date_place
        pug24_dict[s_row["Tag"]]["symposia"][s_group_index][s_id]["Chair"] = s_chairs
        pug24_dict[s_row["Tag"]]["symposia"][s_group_index][s_id]["Mantel"] = s_mantel
        pug24_dict[s_row["Tag"]]["symposia"][s_group_index][s_id]["Talks"] = {}
        
    # Add symposium to session
    abstract_brake = s_row["Abstract"].find("\n")+1
    s_row["Abstract"] = s_row["Abstract"][abstract_brake:]
    abstract_brake = s_row["Abstract"].find("\n\n")
    abstract_title = s_row["Abstract"][:abstract_brake]
    abstract_body = s_row["Abstract"][abstract_brake+2:]
    abstract_brake = abstract_body.rfind("\n\n")
    abstract_body = abstract_body[abstract_brake+2:]
    s_row["Abstract"] = {"title": abstract_title, "body": abstract_body}
    pug24_dict[s_row["Tag"]]["symposia"][s_group_index][s_id]["Talks"][s_index] = s_row.to_dict()

### Save PuG 2024 dict into a JSON file

In [8]:
# Save the dictionary to a JSON file
with open("PuG_2024_data.json", "w") as file:
    json.dump(pug24_dict, file)

### Print symposia to tex files

In [9]:
for day_key, day_value in pug24_dict.items():
    for session_group_key, session_group in day_value["symposia"].items():
        s_id_list = []
        for _, session in session_group.items():

            # cerates a string of all talk items 
            talks = ""
            for _, t in session["Talks"].items():
                if t["Speaker"][-1] == " ": 
                    t["Speaker"] = t["Speaker"][:-1]
                spaker_lName = t["Speaker"].rfind(" ")
                talks += r"""
                \item [%(speaker)s] %(title)s \textcolor{mygray}{ | %(time)s}    
                """ % {
                    "speaker": t["Speaker"][spaker_lName:] + " " + t["Speaker"][0] + ".",
                    "title": t["Abstract"]["title"],
                    "time": t["TalkStart"][:-3]
                }

            # check if chair of multiple unis
            # if session["Chair"]["author"].find(",") != -1 and session["Chair"]["uni"].find(",") != -1:
            #     print(session["Chair"]["author"].split(","))
            #     print(session["Chair"]["uni"].split(","))

            # creates a string with a 'symposium' tex object 
            tex = r"""
            \begin{symposium}
            {%(title)s}
            {%(chairs)s}
            {%(dateplace)s}
            {%(uni)s}
            %(mantel)s
            \begin{description}    
            %(description)s
            \end{description} 
            \end{symposium}
            """ % {
                "title": session["Title"], 
                "chairs": session["Chair"]["author"], 
                "dateplace": session["DatePlace"], 
                "uni": session["Chair"]["uni"], 
                "mantel": session["Mantel"], 
                "description": talks
            }

            # saves the session ID to a list
            # so that a 'file of the files' can be generated
            s_id_list.append(session["Index"])

            # saves the symbosium tex object to a file in the according folder
            f_path = "../booklet/tex/days/" + day_key + "/symposia/"+ session_group_key + "/" + session["Index"] + ".tex"
            f = open(f_path, "w")
            f.write(tex)
            f.close()
        
        session_input = ""
        for id in s_id_list:
            session_input += (r"\input{tex/days/" + day_key + "/symposia/" + session_group_key + "/" + id + ".tex} ")
        f = open("../booklet/tex/days/" + day_key + "/" + session_group_key + ".tex", "w")
        f.write(session_input)
        f.close()
    
